К текущему моменту мы научились обучать простейшую линейную регрессию: причем научились пользоваться как готовыми методами из sklearn, так и реализовывать поиск решения оптимальных коэффициентов самостоятельно с помощью градиентного спуска.

На последних занятиях мы узнали, что на практике часто приходится прибегать к некоторым хитростям, чтобы оценить, действительно ли модель отлично справляется со своей главной задачей - делать хорошие предсказания на новых данных в будущем, или, как еще говорят, в проде - после запуска.

Для этого было введено понятие так называемой обобщающей способности модели. Оно помогает нам оценить, насколько наша модель подогналась под увиденные данные и насколько она окажется хороша на новых данных.

Обобщающую способность мы научились оценивать с помощью метода Кросс-Валидации.



Построим для старой задачи с предсказанием длительности поездки в такси 2 модели линейной регрессии. Для каждой из них замерим качество на кросс-валидации и на тесте, заранее его отложив.

Считается, что если для K-Fold Кросс-Валидации брать большие значения K (то есть большое количество сплитов для разбиения тренировочной выборки на тренировку-валидацию), то оценка среднеквадратической ошибки будет получаться более справедливой. Так, например, когда K оказывается максимальным, а именно равным количеству объектов в выборке (каждый раз в валидации у нас 1 новый объект), то такой метод Кросс-Валидации еще называют LeaveOneOut.

Главная причина, почему мало где его используют, состоит в дороговизне такого метода. Например, в данной задаче в TAXI_train.csv лежит почти 1,2 млн. объектов. Тогда, чтобы замерить качество на LeaveOneOut валидации, нам пришлось бы обучить 1,2 млн. различных версий одной модели, что, конечно, не вписыватеся в рамки адекватного времени и прочих ресурсов.

Поэтому выберем компромиссное значение в виде 20 фолдов для K-Fold Кросс-Валидации.

Внимание! Для каждой из 2 моделей очевидно было бы справедливым замерять качество на тех же самых объектах, поэтому сплиты лучше зафиксировать сразу.

P.S. Оптимизировать будем MSLE метрику. И сравнивать модели тоже будем по ней будем. Как из домашнего задания про метрики, только без корня! :)

$$\text{MSLE}(X, y, a) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2$$


### Модель №1 Для начала посчитаем ошибку на Кросс-Валидации и Тесте для нашей самой базовой модели до вычленения каких-либо признаков, а просто взяв все вещественные колонки.

In [200]:
import pandas as pd

In [201]:
initial_data = pd.read_csv('/Users/alikhansainov/Desktop/Karpov/initial_data.csv', index_col='id')

initial_cols = ['vendor_id', 'passenger_count', 'pickup_longitude',
                'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
                'trip_duration']

initial_data = initial_data[initial_cols]

### ТрюК!

In [202]:
### Замерять будем MSLE. Можно показать, что для оптимизации MSLE,
### Достаточно логарифмировать таргетную переменную, 
### а потом оптимизировать привычные MSE
import numpy as np

initial_data = initial_data.assign(log_trip_duration=np.log1p(initial_data['trip_duration']))
initial_data = initial_data.drop('trip_duration', axis=1)

### Объяснение:

Пусть имеем модель $a(x)$, обученную на MSE от $y$:
$$
\text{MSE}(X, y, a) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(y_i - a(x_i)\big)^2 \rightarrow min
$$

Также представим модель $a^*(x)$, обученную на MSE от $\log{(y + 1)}$:
$$
\text{MSE}(X, \log{(y+1)}, a^*) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - a^*(x_i)\big)^2 \rightarrow min
$$

Так как вторая модель старается аппроксимироваться ответы в виде $a^*(x) \approx \log{(y+1)}$, то для того, чтобы получить изначальные таргеты, необходимо выходы $a^*(x)$ проэкспоненцировать в виде следующего отношения: $a(x) = e^{a^*(x)} - 1$, то есть $a^*(x) = \log{(a(x) + 1)}$, где $a^*(x)$ дает логарифмические выходы, а $a(x)$ - изначальные. Подставим данное соотношение в MSE выше, получим:

$$
\text{MSE}(X, \log{(y+1)}, \log{(a(x) + 1)}) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2 \rightarrow min
$$

А это в точности MSLE от $y$:

$$
\text{MSLE}(X, y, a) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2 \rightarrow min
$$

In [203]:
### Выделим test

from sklearn.model_selection import train_test_split

X = initial_data.drop('log_trip_duration', axis=1)
y = initial_data['log_trip_duration']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

In [204]:
### Применим K-Fold на оставшуюяся валидационную часть X_train, y_train

from sklearn.model_selection import KFold
splitter = KFold(n_splits=5, shuffle=True, random_state=42)
from sklearn.linear_model import LinearRegression
losses_train = []
losses_test = []
for train_fold, test_fold in splitter.split(X_train):
    Xtrain, Xtest = X_train.values[train_fold], X_train.values[test_fold]
    ytrain, ytest = y_train.values[train_fold], y_train.values[test_fold]
    model = LinearRegression()
    model.fit(Xtrain, ytrain)
    losses_train.append(np.mean ((model.predict(Xtrain) - ytrain) ** 2))
    losses_test.append(np.mean ((model.predict(Xtest) - ytest) ** 2))
#print("Train error: ", np.mean(losses_train))
print("Test error: ", np.mean(losses_test))


Test error:  0.6130935871496436


In [205]:
### Теперь построим модель на всей тренировочной выборке
### и замерим качество на тесте!
### Your code is here
from sklearn.metrics import mean_squared_error
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
test_error = mean_squared_error(y_test, y_pred)
test_error


0.6056668879526187

In [206]:
### Укажите в ответе на задание 4 среднее качество моделей на валидационных выборках
### и качество модели, обученной на полной тренировочной выборке, на тестовой выборке.
### В качестве разделителя используйте точку, ответ округлите до тысячных.

### Модель №2. Проделаем все то же самое, только для модели с более осознанными признаками, которые удалось получить ранее

In [207]:
processed_data = pd.read_csv('/Users/alikhansainov/Desktop/Karpov/processed_data copy 2.csv', index_col='id')

In [208]:
### Замерять будем MSLE. Можно показать, что для оптимизации MSLE,
### Достаточно логарифмировать таргетную переменную, 
### а потом оптимизировать привычные MSE
import numpy as np

In [226]:
### Замерьте качество на кросс-валидации!
### Your code is here


from sklearn.model_selection import cross_validate
processed_data = processed_data.assign(log_trip_duration=np.log1p(processed_data['trip_duration']))
X = processed_data.drop(['trip_duration', 'log_trip_duration'], axis=1)
y = processed_data['log_trip_duration']
# X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X,y,test_size=0.2, random_state=42)

test_indexes = X.index
train_indexes = X.index

X_train_2 = X[X.index.isin(train_indexes)]
y_train_2 = y[y.index.isin(train_indexes)]

X_test_2 = X[X.index.isin(test_indexes)]
y_test_2 = y[y.index.isin(test_indexes)]

splitter = KFold(n_splits=20, shuffle=True, random_state=33)
model = LinearRegression()

model.fit(X_train_2, y_train_2)

error = cross_validate(model, X, y, scoring='neg_mean_squared_error', cv=splitter, return_train_score=True)

np.mean(-error['test_score'])


0.4269166886822536

In [227]:
### Теперь построим модель и замерим качество на тесте!
### Your code is here
from sklearn.metrics import mean_squared_error
model = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
test_error = mean_squared_error(y_test, y_pred)
test_error

0.4074669546810213

In [211]:
### Укажите в ответе на задание 5 среднее качество моделей на валидационных выборках
### и качество модели, обученной на полной тренировочной выборке, на тестовой выборке.
### В качестве разделителя используйте точку, ответ округлите до тысячных.



In [212]:
### Найдите оптимальный параметр регуляризации
### для случая Lasso
### Напомним: ошибка на кросс-валидации должно быть 
### строго меньше 0.4

### Шаг №9
### Your code is here
new_data = pd.read_csv('/Users/alikhansainov/Desktop/Karpov/new_data.csv', index_col='id')
X = new_data.drop('log_trip_duration', axis=1)
Y = new_data['log_trip_duration']

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
best_msle = 0.4
selector = KFold(n_splits=4)
alphas = [0.00001,0.001,1.5]
for alpha in alphas:
    scores2 = []
    for train_index, test_index in selector.split(X):
        X_train, X_test = X.values[train_index], X.values[test_index]
        Y_train, Y_test = Y.values[train_index], Y.values[test_index]
        scaler = MinMaxScaler()
        scaler.fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        model_lasso = Lasso(alpha=alpha, max_iter = 100000) 
        model_lasso.fit(X_train_scaled, Y_train)
        predictions = model_lasso.predict(X_test_scaled)
        scores2.append(np.mean((predictions - Y_test)**2))
        mean_msle = np.mean(scores2)
        if mean_msle < best_msle:
            best_msle = mean_msle 
    best_alpha = alpha
    if best_alpha is not None:
        print(f"MSLE на Кросс-валидации равен: {np.mean(scores2)}")
    print (best_alpha)
    break

MSLE на Кросс-валидации равен: 0.39848168595637223
1e-05


In [231]:
def locate_card (cards, query):
    pass
cards = [13,11,10,7,4,3,1,0]
query = 7
output = 3

result = locate_card(cards, query)
print (result)
result == output

test = {
    'input': {
        'cards':[13,11,10,7,4,3,1,0],
        'query':7
    },
    'output':3
}



None


In [232]:
locate_card(**test['input']) == test['output']

False

### Какую модель среди двух стоило бы выбрать? Помогла ли нам базовая обработка признаков с первых уроков? 